In [1]:
# 필수 패키지를 import합니다.
from keras.applications import ResNet50
from keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
from PIL import Image
import numpy as np
import flask
import io

In [2]:
# Flask 애플리케이션과 Keras 모델을 초기화합니다.
app = flask.Flask(__name__)
model = None

첫 번째 코드 조각은 필요한 패키지를 가져오고 Flask 애플리케이션과 Keras 모델을 초기화합니다.

아래는 load_model 함수 정의입니다.

In [3]:
def load_model():
    # 미리 학습된 Keras 모델을 불러옵니다(여기서 우리는 ImageNet으로 학습되고 
    # Keras에서 제공하는 모델을 사용합니다. 하지만 쉽게 하기위해
    # 당신이 설계한 신경망으로 대체할 수 있습니다.)
    global model
    model = ResNet50(weights="imagenet")

함수 이름에서 알 수 있듯이, 이 함수는 신경망을 인스턴스화하고 디스크에서 가중치를 불러오는 역할을 띕니다.

단순하게 하기 위해, ImageNet 데이터 세트로 미리 학삽된 ResNet50 구조를 활용하려 합니다.

클라이언트로부터 오는 데이터를 예측하기 전에, 사전에 데이터를 준비하고 처리하는 과정이 필요합니다.

In [4]:
def prepare_image(image, target):
    # 만약 이미지가 RGB가 아니라면, RGB로 변환해줍니다.
    if image.mode != "RGB":
        image = image.convert("RGB")

    # 입력 이미지 사이즈를 재정의하고 사전 처리를 진행합니다.
    image = image.resize(target)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = imagenet_utils.preprocess_input(image)

    # 처리된 이미지를 반환합니다.
    return image

이 함수는 :

입력 이미지를 받고
(필요하다면) RGB로 이미지를 변환하고
224x224 픽셀 사이즈로 이미지를 재정의하고 (ResNet의 입력 차원에 맞게)
평균 감산과 스케일링을 통해 배열 사전 처리합니다.

다시 말해, 모델을 통해 입력 데이터를 전달하기 전에 필요한 사전 처리, 스케일링, 정규화를 기반으로 함수를 수정해야 합니다.

이제 predict 함수를 정의할 준비가 됐습니다. 이 함수는 /predict 엔트포인트로 어떤 요청들을 처리합니다.

In [5]:
@app.route("/predict", methods=["POST"])
def predict():
    # view로부터 반환될 데이터 딕셔너리를 초기화합니다.
    data = {"success": False}

    # 이미지가 엔트포인트에 올바르게 업로드 되었는디 확인하세요
    if flask.request.method == "POST":
        if flask.request.files.get("image"):
            # PIL 형식으로 이미지를 읽어옵니다.
            image = flask.request.files["image"].read()
            image = Image.open(io.BytesIO(image))

            # 분류를 위해 이미지를 전처리합니다.
            image = prepare_image(image, target=(224, 224))

            # 입력 이미지를 분류하고 클라이언트로부터 반환되는 예측치들의 리스트를 초기화 합니다.
            preds = model.predict(image)
            results = imagenet_utils.decode_predictions(preds)
            data["predictions"] = []

            # 결과를 반복하여 반환된 예측 목록에 추가합니다.
            for (imagenetID, label, prob) in results[0]:
                r = {"label": label, "probability": float(prob)}
                data["predictions"].append(r)

            # 요청이 성공했음을 나타냅니다.
            data["success"] = True

    # JSON 형식으로 데이터 딕셔너리를 반환합니다.
    return flask.jsonify(data)

data 딕셔너리는 클라이언트에게 반환하길 희망하는 데이터를 저장하는데 사용합니다. 이 함수엔 예측의 성공 여부를 나타내는 부울을 가지고 있습니다. 또한, 이 딕셔너리를 사용하여 들어오는 데이터에 대한 예측 결과를 저장합니다.

들어오는 데이터를 승인하기위해 다음 사항을 확인해야 합니다:

요청 방법은 POST(이미지, JSON, 인코딩된 데이터 등을 포함하여 엔트포인트로 임의의 데이터를 보낼수 있도록 함)입니다.
POST를 통해 이미지가 파일 속성으로 전달되었습니다.
그런 다음 그 데이터를 가지고 다음을 진행합니다:

PIL 형식으로 읽어옵니다.
사전 처리를 처리합니다.
신경망을 통해 데이터를 전달합니다.
결과를 반복하고 그 결과들을 각각 data["predictions"]에 추가합니다.
JSON 형태으로 클라이언트에게 응답을 반환합니다.
만약 이미지가 아닌 데이터로 작업한다면, request.files 코드를 삭제하고 원본 입력 데이터를 직접 구문 분석하거나 request.get_json()로 입력 데이터를 python 딕셔너리/객체에 자동으로 구문 분석되도록 해야합니다.

추가로, 참조한 튜토리얼은 Flask의 요청 객체의 기본 요소를 설명하는 내용을 읽어 보세요.

이제 서비스를 시작하겠습니다.

In [ ]:
# 실행에서 메인 쓰레드인 경우, 먼저 모델을 불러온 뒤 서버를 시작합니다.
if __name__ == "__main__":
    print(("* Loading Keras model and Flask starting server..."
        "please wait until server has fully started"))
    load_model()
    app.run()

* Loading Keras model and Flask starting server...please wait until server has fully started
102973440/102967424 [==============================] - 10s 0us/step
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
